In [73]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_page_content(request_url):
# 得到页面的内容
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}
    html=requests.get(request_url,headers=headers,timeout=100).text
    # 创建BeautifulSoup对象
    soup = BeautifulSoup(html, 'html.parser')
    return soup

# 分析当前页面的信息
def analysis(soup):
    temp = soup.find('div', class_="search-result-list")
    # 创建DataFrame
    df = pd.DataFrame(columns=['name','price','picLink'])
    # 提取汽车名字、价格、图片链接
    p_name_list = temp.find_all('p', class_="cx-name text-hover" )
    p_price_list = temp.find_all('p', class_="cx-price" )
    p_piclink_list = temp.find_all('img',class_="img" )
    # print(p_piclink_list)
    # 将结果赋值到df中
    for i in range(len(p_name_list)):
        temp={}
        name, price, picLink = p_name_list[i].text, p_price_list[i].text, p_piclink_list[i].attrs['src']
        temp['name'], temp['price'], temp['picLink'] = name, price, picLink
        df = df.append(temp, ignore_index=True)
    return df

page_num = 3
base_url = 'http://car.bitauto.com/xuanchegongju/?mid=8&page='
    
result = pd.DataFrame(columns=['name','price','picLink'])

# 网页翻页
for i in range(page_num):
    request_url = base_url + str(i+1) 
    # print(request_url)
    soup = get_page_content(request_url)
    df = analysis(soup)
    result = result.append(df)
# print(result)

# DataFrame的index重新排列
result.reset_index(drop=True, inplace=True)

# 将价格按照‘-’进行分列
result[['lowestPrice','highestPrice']] = result['price'].apply(lambda x:pd.Series([i for i in x.split("-")]))
# 生成图片完整地址
result['picLink'] = 'http:'+result['picLink'] 

for i in range(len(result)):
    # 如果result.iloc[i]['lowestPrice']包含中文或已经有“万”字，则不添加万，如果没有，则在结尾加一个“万”
    if u'\u4e00'<= result.iloc[i]['lowestPrice'] <= u'\u9fff':
        continue
    if '万' in result.iloc[i]['lowestPrice']:
        continue
    else:
        # print(result.iloc[i]['lowestPrice'])
        result.loc[i, 'lowestPrice'] = result.loc[i, 'lowestPrice'] + '万'

print(result) 

# 保存.csv格式文件
result.to_csv('Volkswagen_Vehicle_Price.csv', index=False) 

           name         price  \
0            宝来   9.88-15.60万   
1            朗逸   9.99-16.19万   
2            速腾  12.89-19.19万   
3           桑塔纳   8.69-11.59万   
4            迈腾  18.61-30.99万   
5            探岳  18.59-31.39万   
6           高尔夫  14.13-23.42万   
7            凌渡  14.29-19.69万   
8            途岳  16.58-22.38万   
9           帕萨特  18.49-28.29万   
10          途观L  21.88-31.18万   
11           途昂  29.99-50.59万   
12           途铠  11.49-15.99万   
13           探歌  13.58-20.38万   
14      一汽-大众CC  24.68-30.18万   
15           途观  19.48-23.18万   
16           探影  11.49-15.99万   
17          探岳X  23.58-29.58万   
18         Polo   9.99-12.39万   
19          途昂X  28.99-43.99万   
20           途锐  56.98-82.18万   
21           辉昂  34.00-63.90万   
22           威然  28.68-39.98万   
23          途安L  15.18-19.48万   
24           蔚领  12.59-14.97万   
25          甲壳虫  17.88-25.58万   
26       高尔夫·嘉旅  12.88-19.28万   
27      高尔夫(进口)  20.98-37.28万   
28     途观L 插电混动  26.58-32.58万   
29     帕萨特